In [1]:
# import libraries

import pandas as pd
import boto3
from io import StringIO, BytesIO
from datetime import datetime, timedelta
import logging
import os

In [2]:
# define constants

SOURCE_BUCKET = 'xetra-1234'
TARGET_BUCKET = 'xetra-dataset-production'

# setup logging
formatter = ('%(levelname)s:  %(asctime)s:  %(process)s:  %(funcName)s:  %(message)s')
logging.basicConfig(filename="xetra_etl.log", level=logging.DEBUG, force=True, format=formatter, filemode="w")
logger = logging.getLogger()

#stream_handler = logging.StreamHandler()
#file_handler = logging.FileHandler("xetra_etl.log", mode="w")
#file_handler.setFormatter(formatter)
#logger.addHandler(stream_handler)

In [3]:
# Adapter layer - functions to interact with S3

def list_files_in_prefix(bucket, prefix):
    files = [obj.key for obj in bucket.objects.filter(Prefix=prefix)]
    return files

def read_csv_to_df(bucket, key, decoding='utf-8'): #read csv from s3 source to a df
    logger.info(f"Reading file from {bucket}/{key}")
    csv_object = bucket.Object(key=key).get().get('Body').read().decode(decoding)
    data = StringIO(csv_object) #StringIO stores value into binary form
    df = pd.read_csv(data)
    return df

def write_df_to_s3(df, bucket, key):
    output_buffer = BytesIO()
    df.to_parquet(output_buffer, index=False)
    bucket.put_object(Body=output_buffer.getvalue(), Key=key)
    return True
    
def write_df_to_s3_csv(df, bucket, key):
    output_buffer = StringIO()
    df.to_csv(output_buffer, index=False)
    bucket.put_object(Body=output_buffer.getvalue(), Key=key)
    return True

In [4]:
# Application Layer - non-core functions

def return_date_list(bucket, arg_date, src_format, meta_key): #return a list of dates to extract data from
    min_date = datetime.strptime(arg_date, src_format).date() - timedelta(days=1)
    today = datetime.today().date()
    try:
        df_meta = read_csv_to_df(bucket, meta_key)
        dates = [(min_date + timedelta(days=x)) for x in range(0, (today-min_date).days + 1)]
        src_dates = set(pd.to_datetime(df_meta['source_date']).dt.date)
        dates_missing = set(dates[1:]) - src_dates
        if dates_missing:
            min_date = min(set(dates[1:]) - src_dates) - timedelta(days=1)
            return_dates = [date.strftime(src_format) for date in dates if date >= min_date]
            return_min_date = (min_date + timedelta(days=1)).strftime(src_format)
        else:
            return_dates = []
            return_min_date = datetime(2200, 1, 1).date()
    except bucket.session.client('s3').exceptions.NoSuchKey:
        return_dates = [(min_date + timedelta(days=x)).strftime(src_format) for x in range(0, (today-min_date).days + 1)]
        return_min_date = arg_date
    return return_min_date, return_dates

def update_meta_file(bucket, meta_key, extract_date_list):
    old_df = read_csv_to_df(bucket, meta_key)
    new_df = pd.DataFrame(columns=['source_date', 'datetime_of_processing'])
    new_df['source_date'] = extract_date_list
    new_df['datetime_of_processing'] = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    df = pd.concat([old_df, new_df])
    write_df_to_s3_csv(df, bucket, meta_key)
    return df



# Application Layer - core functions for actual ETL operations

def extract(bucket, date_list):
    files = [file for date in date_list for file in list_files_in_prefix(bucket, date)]
    df = pd.concat([read_csv_to_df(bucket, obj) for obj in files], ignore_index=True)
    return df

def transform(df, columns, arg_date):
    df = df.loc[:, columns]
    df.dropna(inplace=True) #drop records with NA
    df['opening_price'] = df.sort_values(by='Time').groupby(['ISIN', 'Date'])['StartPrice'].transform('first') #get opening price per ISIN and day
    df['closing_price'] = df.sort_values(by='Time').groupby(['ISIN', 'Date'])['StartPrice'].transform('last') #get closing price per ISIN and day
    df = df.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), #get aggregations
                                                              closing_price_eur=('closing_price', 'min'), 
                                                              minimum_price_eur=('MinPrice', 'min'), 
                                                              maximum_price_eur=('MaxPrice', 'max'), 
                                                              daily_traded_volume=('TradedVolume', 'sum'))
    df['previous_closing_price'] = df.sort_values(by='Date').groupby('ISIN')['closing_price_eur'].shift(1)
    df['change_previous_closing_%'] = (df['closing_price_eur'] - df['previous_closing_price']) / df['previous_closing_price'] * 100
    df.drop(columns=['previous_closing_price'], inplace=True)
    df = df.round(decimals=2)
    df = df[df.Date >= arg_date]
    return df

def load(df, bucket, arg_date, src_format, trg_format, trg_key, meta_key, extract_date_list):
    arg_date_dt = datetime.strptime(arg_date, src_format).date() - timedelta(days=1)
    key = trg_key + datetime.today().strftime('%Y%m%d_%H%M%S') + trg_format
    write_df_to_s3(df, bucket, key)
    update_meta_file(bucket, meta_key, extract_date_list)
    return True

def etl(src_bucket, trg_bucket, date_list, columns, src_format, arg_date, trg_format, trg_key, meta_key):
    df = extract(src_bucket, date_list)
    df = transform(df, columns, arg_date)
    extract_date_list = [date for date in date_list if date >= arg_date]
    return load(df, trg_bucket, arg_date, src_format, trg_format, trg_key, meta_key, extract_date_list)

In [5]:
# Main function - entrypoint to application

def main():
    arg_date = '2022-12-06' #minimum date to process
    columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
    src_format = '%Y-%m-%d'
    trg_format = '.parquet'
    trg_key = 'xetra_daily_report_'
    meta_key = 'meta_file.csv'
    
    #initialize connection to s3 buckets
    s3 = boto3.resource('s3')
    src_bucket = s3.Bucket(SOURCE_BUCKET)
    trg_bucket = s3.Bucket(TARGET_BUCKET)
    
    #return objects from source
    extract_date, date_list = return_date_list(trg_bucket, arg_date, src_format, meta_key)
    
    #execute etl
    etl(src_bucket, trg_bucket, date_list, columns, src_format, extract_date, trg_format, trg_key, meta_key)

In [6]:
# run application
main()

## Read output file

In [7]:
s3 = boto3.resource('s3')
trg_bucket = s3.Bucket(TARGET_BUCKET)

for obj in trg_bucket.objects.all():
    print(obj.key)

meta_file.csv
xetra_daily_report_2022-12-08_13:28:20.parquet
xetra_daily_report_2022-12-08_17:46:30.parquet
xetra_daily_report_20221208_085101.parquet
xetra_daily_report_20221208_085649.parquet
xetra_daily_report_20221208_095001.parquet
xetra_daily_report_20221208_112350.parquet
xetra_daily_report_20221208_124139.parquet
xetra_daily_report_20221208_125355.parquet
xetra_daily_report_20221208_132515.parquet
xetra_daily_report_20221208_174950.parquet
xetra_daily_report_20221211_140529.parquet


In [8]:
parquet_object = trg_bucket.Object(key='xetra_daily_report_20221208_125355.parquet').get().get('Body').read()
data = BytesIO(parquet_object)
df_report = pd.read_parquet(data)

df_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_previous_closing_%
0,AT000000STR1,2022-12-08,38.75,38.75,38.50,38.75,251,1.31
1,AT00000FACC2,2022-12-08,9.10,9.17,9.10,9.30,3906,0.44
2,AT0000606306,2022-12-08,26.30,27.58,26.30,27.58,6789,4.87
3,AT0000609607,2022-12-08,12.96,12.96,12.96,12.96,0,1.57
4,AT0000644505,2022-12-08,106.60,106.60,106.60,106.60,0,1.52
...,...,...,...,...,...,...,...,...
3216,XS2265370234,2022-12-08,19.26,19.94,19.26,19.94,13,1.02
3217,XS2284324667,2022-12-08,36.98,36.53,36.53,36.98,3500,-1.35
3218,XS2314659447,2022-12-08,8.56,8.56,8.51,8.56,90,-0.08
3219,XS2314660700,2022-12-08,20.57,19.99,19.99,20.57,0,-3.72


In [9]:
# read metadata file

s3 = boto3.resource('s3')
trg_bucket = s3.Bucket(TARGET_BUCKET)
meta_key = 'meta_file.csv'

meta_file = read_csv_to_df(trg_bucket, meta_key)

meta_file

,source_date,datetime_of_processing
0,12/6/2022,2022-12-06 12:33:23
1,12/7/2022,2022-12-07 12:30:21
2,2022-12-08,2022-12-08_17:49:52
3,2022-12-09,2022-12-11 14:05:32
4,2022-12-10,2022-12-11 14:05:32
5,2022-12-11,2022-12-11 14:05:32
